In [1]:
from toolz_graph import *


In [2]:
import sys
import json
import pprint
from toolz.curried import *

The introspect-data-linux contains data from a parsed .tu file and cleaned up with scripts to normalize it. Later we will emit that clean json directly but for now it is the interface used.

In [3]:
input_file = open("introspector-data-linux/linux_clean.json")
nodes = json.load(input_file)['nodes']

An example node, the _id is the node id, _type is the type of the field.
The other attributes are either refrences to nodes or literals, there is no denotation of the type of the field here, but the fieldnames are clearly defined.  

In [4]:
def field_report(data):
    """Helper function to print out the fields in order of usage"""
    field_names_data = dict(frequencies(list(concat(map(keys, data)))))
    items = field_names_data.items()
    sorted_fields = list(map(first, list(reversed(sorted(items, key=nth(1))))))
    return sorted_fields


In [5]:
pprint.pprint(nodes[0])

{'_id': '1', '_type': 'type_decl', 'chain': '4', 'name': '2', 'type': '3'}


In [6]:
field_report(nodes)


['_type',
 '_id',
 'type',
 'name',
 'srcp',
 'scpe',
 'OP0 :',
 'chain',
 'algn',
 'size',
 '_string_len',
 '_string',
 'valu',
 'chan',
 'OP1',
 'body',
 'link',
 'bpos',
 'retn',
 'prms',
 'used',
 'E0',
 'purp',
 'cnst',
 'E1',
 'fn',
 'value',
 'ptd',
 'argt',
 'E2',
 'expr',
 'tag',
 'unql',
 'flds',
 'mngl',
 'prec',
 'sign',
 'min',
 'max',
 'args',
 'labl',
 'E3',
 'init',
 'val',
 'vars',
 'elts',
 'domn',
 'E4',
 'qual',
 'E5',
 'csts',
 'OP2',
 'E6',
 'E7',
 'E8',
 'E9',
 'E10',
 'low',
 'E11',
 'E12',
 'decl',
 'E13',
 'E14',
 'E15',
 'idx',
 'E16',
 'E17',
 'E18',
 'E19',
 'E20',
 'E21',
 'E22',
 'E23',
 'E25',
 'E24',
 'E26',
 'E27',
 'E28',
 'E29',
 'E31',
 'E30',
 'E35',
 'E34',
 'E33',
 'E32',
 'E39',
 'E38',
 'E37',
 'E36',
 'cond',
 'E43',
 'E42',
 'E41',
 'E40',
 'E47',
 'E46',
 'E45',
 'E44',
 'E54',
 'E53',
 'E52',
 'E51',
 'E50',
 'E49',
 'E48',
 'E62',
 'E61',
 'E60',
 'E59',
 'E58',
 'E57',
 'E56',
 'E55',
 'refd',
 'E97',
 'E96',
 'E95',
 'E94',
 'E93',
 'E92

In [7]:
def join_field(role_from, role_to, field_name, nodes, exclude_left=(), exclude_right=()):
    from_field_name = field_name
    if role_from:
        from_field_name = role_from + "_" + field_name
    to_field_name = role_to + "__id"
    
    results = keyjoin(
            from_field_name,
            filterfalse(
                get_in(field_name),
                map(
                    pushdown(role_from, field_name, exclude_left),
                    filter(_hasattr(field_name), nodes),
                ),
            ),
            to_field_name,
            map(pushdown(role_to, "_id", exclude_right), nodes),
        )
    # leave in all the ids
    # results = map(mdissoc((to_field_name, from_field_name)),results)
        
    return list(results)

Now we have a list of nodes we can join the nodes with themselves on the field named `type`. We name things on the left typed and things on the right the `typed_type` meaning is is the type behind the typed object. This will give us a nice table structure

In [8]:
types_list = join_field("typed",
                        "typed_type",
                        'type',
                         nodes)

NameError: name '_hasattr' is not defined

In [ ]:
names = join_field("named",
                        "name",
                        'name',
                         nodes)

In [ ]:
field_report(names)

In [ ]:
string_names= [x for x in names if 'name__string' in x]
string_names

This next block joins the names of things where the name itself is not a string,
some names type decls and the name of them is the actual string .
The actual string is represented here in the field 'name_name__string'
 

In [ ]:

names2 = join_field_extra(
            role_from=None,
            role_to="name",
            from_field_name="name_name",        
            to_field_name='_id',
            nodes_from=names,
            nodes_to=nodes,
            debug=True,
            exclude_right = (
                'type',
                '_type',
            )
        )

Here is a unique list of the strings used in the names

In [ ]:
set(map(get('name__string'),names2))

In [ ]:
scpe_list = join_field("scoped",
                        "scope",
                        'scpe',
                         nodes)
field_report(scpe_list)

Now we can merge the simple and complex names together into one list for lookup

In [ ]:
all_names = string_names + names2
sorted(field_report(all_names))

In [ ]:

def filter_nodes(role, field_name, nodes, excludes) :
    return list(filterfalse(
                get_in(field_name),
                map(
                    pushdown(role, field_name, excludes),
                    filter(_hasattr(field_name), nodes),
                ),
            ))
    
def join_field_extra(
    role_from, from_field_name, nodes_from,
    role_to,  to_field_name, nodes_to,
                     debug=True,
                     exclude_left=(),
                     exclude_right=()):
    """The new from field name  is the field that is generated in the new obj
the new to field is also prefixed by the role
    """
    new_from_field_name = from_field_name
    if role_from:
        new_from_field_name = role_from + "_" + from_field_name
    new_to_field_name = to_field_name
    if role_to :
        new_to_field_name = role_to + "_" + to_field_name
 
    left = filter_nodes(role_from, from_field_name, nodes_from, exclude_left)
    right =filter_nodes(role_to, to_field_name, nodes_to, exclude_right) 

    if debug:
        if left:
            pprint.pprint({
                "left examples" :left[0:3],
                "new_left_key" : new_from_field_name,
                "left_key" : from_field_name,
            })
        if right:
            pprint.pprint({
                 "right example" : right[0:3],
                "right_key" :to_field_name,
                "new_right_key" :new_to_field_name})
    if not right :
        raise Exception()
    if not left:
        raise Exception()
    results = keyjoin(
            new_from_field_name,
            left,
            new_to_field_name,
            right,
        )
    # results = map(mdissoc((new_to_field_name, new_from_field_name)),results)
    results_list = list(results)

    if debug:
        pprint.pprint({
            'results-len': len(results_list),
         })
        if len(results_list):
          pprint.pprint({
            'results-example' : results_list[0:3]})
    
    return results_list


In [ ]:
field_report(all_names)

In [ ]:
#field_report(all_names)
scope_name = join_field_extra(
            nodes_from=scpe_list,
            role_from=None,
            from_field_name='scope_name',
            role_to='scope_name',
    nodes_to=all_names,        
    to_field_name='name__id',            
    debug=True)
field_report(scope_name)


In [ ]:
scoped_name = join_field_extra(
            nodes_from=scpe_list,
            role_from=None,
            from_field_name='scoped_name',
            role_to='scoped_name',
    nodes_to=all_names,        
    to_field_name='name__id',                
    debug=True)
field_report(scoped_name)

now we join the scoped with the scope names

In [15]:
scoped_scope = join_field_extra(
            nodes_from=scoped_name,
            role_from='left',
            from_field_name='scope__id',
         
    
            role_to='right',
            nodes_to=all_names,        
            to_field_name='named__id',                
    debug=True)

NameError: name 'scoped_name' is not defined

now show some names

In [ ]:
simple_report = list({ 
   obj['left_scoped_name_named__type'] : obj['left_scoped_name_name__string'],
    obj['right_named__type'] : obj['right_name__string'],
} for obj in list(scoped_scope))

field_report(simple_report)

This next report shows you the contained object and the container object by type, we see a good distribution of objects

In [10]:
simple_report2 = groupby(
   [ 'left_scoped_name_named__type' , 
    'right_named__type'] , list(scoped_scope))
set(simple_report2)

NameError: name 'scoped_scope' is not defined

In [ ]:
fields

In [ ]:
from IPython.display import HTML, display

In [11]:

# fields = field_report(scoped_scope)
fields = string_fields

header = '</th><th>'.join(fields)

display(HTML(
   '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format(
       header, 
       '</tr><tr>'.join(
        '<td>{}</td>'.format('</td><td>'.join(str(row.get(k,'null')) for k in fields)) for row in scoped_scope)
       )))

NameError: name 'string_fields' is not defined

In [12]:
 [x for x in field_report(scoped_scope) if x.endswith('string') or '__type' in x]


NameError: name 'scoped_scope' is not defined